# Week 4 - logistic regression and feature scaling

In [8]:
#pip install kaggle

In [5]:
import os
import json

# Replace this with the content of your kaggle.json
kaggle_token = {
    "username": "your_kaggle_username",
    "key": "your_kaggle_api_key"
}

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(kaggle_token, f)

os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)
import os

# Make sure your kaggle.json is correctly set up
os.environ['KAGGLE_CONFIG_DIR'] = os.path.expanduser("~/.kaggle")

# Download the dataset
!kaggle datasets download -d ahmedshahriarsakib/usa-real-estate-dataset

# Unzip the dataset
!unzip -o usa-real-estate-dataset.zip -d usa_real_estate



Dataset URL: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset
License(s): other
  0%|                                               | 0.00/38.2M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 38.2M/38.2M [00:00<00:00, 1.08GB/s]
Archive:  usa-real-estate-dataset.zip
  inflating: usa_real_estate/realtor-data.zip.csv  


In [1]:
import pandas as pd

df = pd.read_csv('usa_real_estate/realtor-data.zip.csv')  
print(df.head())

   brokered_by    status     price  bed  bath  acre_lot     street  \
0     103378.0  for_sale  105000.0  3.0   2.0      0.12  1962661.0   
1      52707.0  for_sale   80000.0  4.0   2.0      0.08  1902874.0   
2     103379.0  for_sale   67000.0  2.0   1.0      0.15  1404990.0   
3      31239.0  for_sale  145000.0  4.0   2.0      0.10  1947675.0   
4      34632.0  for_sale   65000.0  6.0   2.0      0.05   331151.0   

         city        state  zip_code  house_size prev_sold_date  
0    Adjuntas  Puerto Rico     601.0       920.0            NaN  
1    Adjuntas  Puerto Rico     601.0      1527.0            NaN  
2  Juana Diaz  Puerto Rico     795.0       748.0            NaN  
3       Ponce  Puerto Rico     731.0      1800.0            NaN  
4    Mayaguez  Puerto Rico     680.0         NaN            NaN  


In [2]:
austin_df = df[(df['city'] == 'Austin') & (df['state'] == 'Texas')]
austin_df = austin_df[['price', 'bed', 'bath', 'acre_lot', 'house_size']].dropna()
austin_df

,price,bed,bath,acre_lot,house_size
1119516,500000.0,3.0,2.0,0.16,1320.0
1119518,650000.0,2.0,2.0,0.09,870.0
1119522,350000.0,2.0,1.0,0.03,873.0
1119523,1345000.0,4.0,4.0,0.36,3496.0
1119524,599000.0,3.0,2.0,0.16,1484.0
...,...,...,...,...,...
1991922,559000.0,6.0,4.0,0.36,2927.0
1991923,579000.0,3.0,2.0,0.22,1465.0
1991924,830000.0,4.0,3.0,0.21,3220.0
1991925,399900.0,3.0,2.0,0.18,1505.0


In [3]:
from sklearn.preprocessing import StandardScaler
X = austin_df[['bed', 'bath', 'acre_lot', 'house_size']]
y = austin_df['price']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
X = austin_df[['bed', 'bath', 'acre_lot', 'house_size']]

# Choose either normalization or standardization:
scaler = StandardScaler()  # or MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [6]:
from sklearn.preprocessing import MinMaxScaler
#another try
scaler2 = MinMaxScaler()  
X_scaled_2 = scaler2.fit_transform(X)

In [8]:
# Re-run code after environment reset to load and process the dataset again
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import os


# Filter for Austin, Texas and clean
austin_df = df[(df['city'] == 'Austin') & (df['state'] == 'Texas')]
austin_df = austin_df[['price', 'bed', 'bath', 'acre_lot', 'house_size']].dropna()

# Create a binary target: price above median or not
median_price = austin_df['price'].median()
austin_df['target'] = (austin_df['price'] > median_price).astype(int)

# Features and target
X = austin_df[['bed', 'bath', 'acre_lot', 'house_size']]
y = austin_df['target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Logistic Regression with StandardScaler
scaler_std = StandardScaler()
X_train_std = scaler_std.fit_transform(X_train)
X_test_std = scaler_std.transform(X_test)

model_std = LogisticRegression(max_iter=1000)
model_std.fit(X_train_std, y_train)
y_pred_std = model_std.predict(X_test_std)
accuracy_std = accuracy_score(y_test, y_pred_std)
report_std = classification_report(y_test, y_pred_std, output_dict=True)

# 2. Logistic Regression with MinMaxScaler
scaler_minmax = MinMaxScaler()
X_train_mm = scaler_minmax.fit_transform(X_train)
X_test_mm = scaler_minmax.transform(X_test)

model_mm = LogisticRegression(max_iter=1000)
model_mm.fit(X_train_mm, y_train)
y_pred_mm = model_mm.predict(X_test_mm)
accuracy_mm = accuracy_score(y_test, y_pred_mm)
report_mm = classification_report(y_test, y_pred_mm, output_dict=True)

# Combine results for comparison
comparison = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision (0)', 'Recall (0)', 'F1-score (0)', 'Precision (1)', 'Recall (1)', 'F1-score (1)'],
    'StandardScaler': [
        accuracy_std,
        report_std['0']['precision'],
        report_std['0']['recall'],
        report_std['0']['f1-score'],
        report_std['1']['precision'],
        report_std['1']['recall'],
        report_std['1']['f1-score']
    ],
    'MinMaxScaler': [
        accuracy_mm,
        report_mm['0']['precision'],
        report_mm['0']['recall'],
        report_mm['0']['f1-score'],
        report_mm['1']['precision'],
        report_mm['1']['recall'],
        report_mm['1']['f1-score']
    ]
})


In [9]:
comparison

,Metric,StandardScaler,MinMaxScaler
0,Accuracy,0.785479,0.762376
1,Precision (0),0.765823,0.727811
2,Recall (0),0.812081,0.825503
3,F1-score (0),0.788274,0.773585
4,Precision (1),0.806897,0.805970
5,Recall (1),0.759740,0.701299
6,F1-score (1),0.782609,0.750000


Both perform well; standard scaling slightly better.StandardScaler is more precise when predicting “below median” homes 

StandardScaler performs slightly better overall, especially for identifying expensive homes (class 1), which might be more valuable in real estate targeting

The difference isn't massive — this suggests both scalers make the data suitable for logistic regression, but StandardScaler leads to more stable, balanced results in this case